In [2]:
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import mlflow
from mlflow.models.signature import infer_signature
from refactoring import data_loading, preprocess, mlflow_helper, evaluation
# Start the MLflow server
mlflow_helper.start_mlflow()
# Setup MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("wikipedia")

Docker Desktop is already running.
MLflow is already running.


<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1729384150209, experiment_id='3', last_update_time=1729384150209, lifecycle_stage='active', name='wikipedia', tags={}>

In [3]:
# Setup training data

columns = ['E_NEds', 'E_Bpag', 'NEds', 'NDays', 'NActDays',
           'NPages', 'NPcreated', 'pagesWomen', 'wikiprojWomen', 'weightIJ', 'NIJ']
#columns = ["C_api", "C_man", "E_NEds", "E_Bpag", "firstDay", "lastDay", "NEds", "NDays", "NActDays", "NPages", "NPcreated", "pagesWomen", "wikiprojWomen", "ns_user", "ns_wikipedia", "ns_talk", "ns_userTalk", "ns_content", "weightIJ", "NIJ"]
X_train, X_test, y_train, y_test = preprocess.get_data(columns)


NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [3]:
# Random Forest model
mlflow.sklearn.autolog(log_post_training_metrics=False)
with mlflow.start_run() as run:
    rf_model = Pipeline(steps=[
        ('random_forest', RandomForestClassifier(bootstrap = True, max_depth = None, max_features = 'sqrt', min_samples_leaf = 2, min_samples_split = 2, n_estimators = 200, random_state=42))
    ])
    rf_model.fit(X_train, y_train)
    accuracy = rf_model.score(X_test, y_test)
    mlflow.log_metric("accuracy", accuracy)

    # Log the model to MLflow
    mlflow.sklearn.log_model(
        sk_model=rf_model,
        artifact_path="model",
        signature=infer_signature(X_train, rf_model.predict(X_train)),
        input_example= X_train.head(1))


2024/10/20 01:37:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run victorious-sow-167 at: http://localhost:5000/#/experiments/3/runs/1c42589fb6004f7a8c56e00bf4fb06e5.
2024/10/20 01:37:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


In [4]:

loaded_model,last_run = mlflow_helper.load_last_mlflow_model("http://localhost:5000", "wikipedia")
evaluation.evaluate_model(loaded_model, X_test, y_test, last_run)

Latest run ID: 1c42589fb6004f7a8c56e00bf4fb06e5


Error Cuadrático Medio (MSE) - RandomForestClassifier: 0.0
Accuracy - RandomForestClassifier: 1.0
Informe de clasificación - RandomForestClassifier:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       307
           1       1.00      1.00      1.00       528
           2       1.00      1.00      1.00        68

    accuracy                           1.00       903
   macro avg       1.00      1.00      1.00       903
weighted avg       1.00      1.00      1.00       903



In [5]:
# Logistic Regression model
with mlflow.start_run() as run:
    logistic_model = Pipeline(steps=[
        ('logistic_regression', LogisticRegression(random_state=42,solver='lbfgs',multi_class='multinomial'))
    ])

    logistic_model.fit(X_train, y_train)
    logistic_accuracy = logistic_model.score(X_test, y_test)
    mlflow.log_metric("logistic_accuracy", logistic_accuracy)

    # Log the model to MLflow
    mlflow.sklearn.log_model(
        sk_model=logistic_model,
        artifact_path="model",
        signature=infer_signature(X_train, rf_model.predict(X_train)),
        input_example= X_train.head(1))

c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


2024/10/20 01:37:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run dashing-colt-945 at: http://localhost:5000/#/experiments/3/runs/1cf8a9ab7af141748e22a331f6c6dd64.
2024/10/20 01:37:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/3.


In [6]:
loaded_model, last_run = mlflow_helper.load_last_mlflow_model("http://localhost:5000", "wikipedia")
evaluation.evaluate_model(loaded_model, X_test, y_test, last_run)

Latest run ID: 1cf8a9ab7af141748e22a331f6c6dd64


Error Cuadrático Medio (MSE) - LogisticRegression: 0.0
Accuracy - LogisticRegression: 1.0
Informe de clasificación - LogisticRegression:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       307
           1       1.00      1.00      1.00       528
           2       1.00      1.00      1.00        68

    accuracy                           1.00       903
   macro avg       1.00      1.00      1.00       903
weighted avg       1.00      1.00      1.00       903

